<a href="https://colab.research.google.com/github/gwohlgen/colab/blob/master/SW_Course_2019_unit10_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install SPARQLWrapper

     |████████████████████████████████| 348kB 9.6MB/s 
     |████████████████████████████████| 51kB 27.4MB/s 


http://dbpedia.org/sparql

In [0]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")

data = []

for category in ('Athlete', 'Politician', 'Writer'):
  print('\n\nNew category::', category)
  
  sparql.setQuery("""
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX dbo: <http://dbpedia.org/ontology/>
      PREFIX dbp: <http://dbpedia.org/resource/>
      PREFIX foaf: <http://xmlns.com/foaf/0.1/>

      SELECT ?p ?type ?abstract
      WHERE { 

      ?p rdf:type dbo:%s .
      ?p dbo:abstract ?abstract .

      FILTER(LANG(?abstract)='en')
      } LIMIT 2000
  """ % (category,)) 

  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()


  for result in results["results"]["bindings"]:
      print(result["p"]["value"])
      print(result["abstract"]["value"])
      
      output = "__label__" + category + '  ' + result["abstract"]["value"]
      
      data.append(output)



New category:: Athlete
http://dbpedia.org/resource/Siniša_Žugić
Siniša Žugić (Serbian: Синиша Жугић) (born December 17, 1969) is a Serbian diver. He competed at the 1996 Summer Olympics in Atlanta and took 38th place in Men's Springboard. He currently works as a coach in Vračar Diving Club.
http://dbpedia.org/resource/Witold_Gerutto
Witold Gerutto (1 October 1912 in Harbin, China – 13 October 1973 in Konstancin) was a Polish shot putter and decathlete who competed in the 1948 Summer Olympics.
http://dbpedia.org/resource/Antun_Herceg
Antun Herceg (Serbian Cyrillic: Aнтун Xepцeг; born November 9, 1927 in Inđija, Kingdom of Yugoslavia) is a former Serbian football player.
http://dbpedia.org/resource/Daryl_Homer
Daryl Homer (born July 16, 1990) is an American Olympic saber fencer. He competed in the 2012 Summer Olympics and the 2016 Summer Olympics. He won the silver medal in individual saber at the 2016 Olympics. He was also a silver medalist at the 2015 World Fencing Championships, and

In [0]:

import random
random.shuffle(data)
from pprint import pprint
#pprint(data[:3])
print(len(data))


## train dev test split
train = data[:int(len(data)*0.6)]
dev = data[int(len(data)*0.6):int(len(data)*0.8)]
test = data[int(len(data)*0.8):]

print(len(train), len(test), len(dev))

fh = open('train.txt', 'w')
for line in train:
  fh.write(line+'\n')
  
fh = open('dev.txt', 'w')
for line in dev:
  fh.write(line+'\n')
  
fh = open('test.txt', 'w')
for line in test:
  fh.write(line+'\n')


6000
3600 1200 1200


In [0]:
!pip install flair

In [0]:
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer


In [0]:
!wc *.txt
!head dev.txt

   121  16512 101935 dev.txt
   115  15725  97081 test.txt
   360  47506 294838 train.txt
   596  79743 493854 total
__label__Writer  Akinwande Oluwole "Wole" Babatunde Soyinka (Yoruba: Akinwándé Oluwo̩lé Babátúndé S̩óyinká, pronounced [wɔlé ʃójĩŋká]; born 13 July 1934) is a Nigerian playwright and poet. He was awarded the 1986 Nobel Prize in Literature, the first African to be honored in that category. Soyinka was born into a Yoruba family in Abeokuta. After studying in Nigeria and the UK, he worked with the Royal Court Theatre in London. He went on to write plays that were produced in both countries, in theatres and on radio. He took an active role in Nigeria's political history and its struggle for independence from Great Britain. In 1965, he seized the Western Nigeria Broadcasting Service studio and broadcast a demand for the cancellation of the Western Nigeria Regional Elections. In 1967 during the Nigerian Civil War, he was arrested by the federal government of General Yakubu Gow

In [0]:
corpus: TaggedCorpus = NLPTaskDataFetcher.load_classification_corpus('.',
                                                                     test_file='test.txt',
                                                                     dev_file='dev.txt',
                                                                     train_file='train.txt')


2019-05-15 08:35:27,576 Reading data from .
2019-05-15 08:35:27,579 Train: train.txt
2019-05-15 08:35:27,580 Dev: dev.txt
2019-05-15 08:35:27,582 Test: test.txt


In [0]:
print(corpus.obtain_statistics())

{
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 3600,
        "number_of_documents_per_class": {
            "Politician": 1183,
            "Athlete": 1228,
            "Writer": 1189
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 397425,
            "min": 8,
            "max": 1696,
            "avg": 110.39583333333333
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 1196,
        "number_of_documents_per_class": {
            "Writer": 407,
            "Politician": 407,
            "Athlete": 382
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 136684,
            "min": 4,
            "max": 1288,
            "avg": 114.28428093645485
        }
    },
    "DEV": {
        "dataset": "DEV",
        "total_number_of_documents": 1200,
        "number_of_documents_per_class": {
            "Athlete

In [0]:
label_dict = corpus.make_label_dictionary()
label_dict.item2idx

{b'Athlete': 2, b'Politician': 0, b'Writer': 1}

In [0]:
word_embeddings = [WordEmbeddings('glove'),

                   # comment in flair embeddings for state-of-the-art results
                   FlairEmbeddings('news-forward'),
                   FlairEmbeddings('news-backward'),
                   ]

2019-05-15 08:24:11,857 this function is deprecated, use smart_open.open instead
2019-05-15 08:24:13,499 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4.1/big-news-forward--h2048-l1-d0.05-lr30-0.25-20/news-forward-0.4.1.pt not found in cache, downloading to /tmp/tmppom63bp8


100%|██████████| 73034624/73034624 [00:04<00:00, 17041903.90B/s]

2019-05-15 08:24:18,310 copying /tmp/tmppom63bp8 to cache at /root/.flair/embeddings/news-forward-0.4.1.pt


2019-05-15 08:24:18,395 removing temp file /tmp/tmppom63bp8
2019-05-15 08:24:19,127 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4.1/big-news-backward--h2048-l1-d0.05-lr30-0.25-20/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmpyelrvadq


100%|██████████| 73034575/73034575 [00:04<00:00, 17607776.28B/s]

2019-05-15 08:24:23,815 copying /tmp/tmpyelrvadq to cache at /root/.flair/embeddings/news-backward-0.4.1.pt


2019-05-15 08:24:23,892 removing temp file /tmp/tmpyelrvadq


In [0]:
# 4. initialize document embedding by passing list of word embeddings
# Can choose between many RNN types (GRU by default, to change use rnn_type parameter)
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                     hidden_size=512,
                                                                     reproject_words=True,
                                                                     reproject_words_dimension=256,
                                                                     )

# 5. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, multi_label=False)



In [0]:

# 6. initialize the text classifier trainer
trainer = ModelTrainer(classifier, corpus)

# 7. start the training
trainer.train('classif',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=5,
              max_epochs=4)


2019-05-15 08:36:01,252 ----------------------------------------------------------------------------------------------------
2019-05-15 08:36:01,254 Evaluation method: MICRO_F1_SCORE
2019-05-15 08:36:01,256 ----------------------------------------------------------------------------------------------------
2019-05-15 08:36:04,545 epoch 1 - iter 0/113 - loss 0.03395006
2019-05-15 08:36:43,960 epoch 1 - iter 11/113 - loss 0.03295929
2019-05-15 08:37:21,106 epoch 1 - iter 22/113 - loss 0.03138715
2019-05-15 08:38:05,359 epoch 1 - iter 33/113 - loss 0.02899607
2019-05-15 08:38:37,660 epoch 1 - iter 44/113 - loss 0.02690302
2019-05-15 08:39:28,740 epoch 1 - iter 55/113 - loss 0.02563785
2019-05-15 08:40:06,859 epoch 1 - iter 66/113 - loss 0.02544348
2019-05-15 08:40:37,187 epoch 1 - iter 77/113 - loss 0.02406869
2019-05-15 08:41:08,268 epoch 1 - iter 88/113 - loss 0.02329143
2019-05-15 08:41:47,626 epoch 1 - iter 99/113 - loss 0.02205719
2019-05-15 08:42:25,345 epoch 1 - iter 110/113 - loss

In [0]:
from flair.data import Sentence

sentence = Sentence('Patrick wrote about the meaning of life in 3 books.')

# predict tags and print
classifier.predict(sentence)

print(sentence.labels)

[Writer (0.62239670753479)]


[Sentence: "Patrick wrote about the meaning of life." - 7 Tokens]